In [62]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
import pickle
print('Versão do TensorFlow: ', tf.__version__)
import platform

Versão do TensorFlow:  2.16.1


In [63]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [64]:
os = platform.system()
path = './angle_datas.pkl'

log_dir = "../../logs/horizontal_direction_datas/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_model = "./models/horizontal_direction_datas/model_saved.h5"
save_model_keras = "./models/horizontal_direction_datas/model_saved.keras"



if os == 'Darwin':  # 'Darwin' é o nome do kernel do macOS
    path = '../../datas/angle_datas.pkl'
    save_model = "../../models/horizontal_direction_datas/model_saved.h5"
    save_model_keras = "../../models/horizontal_direction_datas/model_saved.keras"

In [65]:

# Carregar os dados do arquivo .pkl
with open(path, 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
    
    
 

In [68]:
%load_ext tensorboard
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(2,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(2, activation='softmax')  #  
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=10000, validation_split=0.2, callbacks=[tensorboard_callback])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/100


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_5_1/Cast:0", shape=(None,), dtype=float32). Expected shape (None, 2), but input has incompatible shape (None,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None,), dtype=int64)
  • training=True
  • mask=None

In [ ]:
# Avaliar o modelo
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Model Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Ações
# 0 Go to left
# 1 Go to Right

# GO TO RIGHT
targetX=100
agentX = 0
class_names = [  'Go to left', 'Go to Right', ]
predictions = model.predict(np.array([[targetX,agentX]]))
 
score = tf.nn.softmax(predictions[0])
print(  "{}-Must be GO TO RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

# GO TO Left
targetX=0
agentX = 100
 
predictions = model.predict(np.array([[targetX,agentX]]))
 
score = tf.nn.softmax(predictions[0])
print(  "{}-Must be GO TO LETH: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)
 



In [ ]:
%tensorboard

In [ ]:
model.save(save_model)
model.save(filepath=save_model_keras)

 

### From here to you only MACOS to test 

In [ ]:
# !pip install tensorflowjs
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('../../models/horizontal_direction_datas/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, '../../models/horizontal_direction_datas/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model '../../models/horizontal_direction_datas/saved_model/' ../../models/tf/horizontal_direction_datas/web_model